In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
def list_append(value, list_name):
    if value != "N/A":
        list_name.append(value)
    else: 
        list_name.append('Null')

In [3]:
df1 = pd.read_excel('./2019.xlsx', sheet_name='연결재무재표')
df2 = pd.read_excel('./2019.xlsx', sheet_name='포괄손익계산서_연결')

df1_cols = ['회사명', '종목코드', '결산월', '항목명', '당기', '전기', '전전기']
df2_cols = ['회사명', '종목코드', '결산월', '항목명', '당기', '전기', '전전기']

df3 = df1[df1_cols]
df4 = df2[df2_cols]

df_ts = df4[df4['항목명']=='매출총이익']
df_tm = df3[df3['항목명']=='   자본총계']

result = pd.merge(df_ts, df_tm, on=['회사명'])
result['magic']=result['당기_x']/result['당기_y']
magic = result[['회사명', 'magic']]
magic = magic[magic['magic']>0]
magic = magic.sort_values('magic', axis=0, ascending=False)
magic['magic_rank'] = magic['magic'].rank(ascending=False, method='max')

"""table1 = df3[df3['항목명']=='   부채총계']
table2 = df3[df3['항목명']=='   자본총계']
table3 = pd.merge(table1, table2, on='종목코드')

table3['부채비율']=table3['당기_x']/table3['당기_y']*100
table3[table3['부채비율']<100]

df_sales1 = df4[df4['항목명']=='영업이익']
df_sales2 = df4[df4['항목명']=='영업이익(손실)']
df_sales = df_sales1.append(df_sales2)"""

"table1 = df3[df3['항목명']=='   부채총계']\ntable2 = df3[df3['항목명']=='   자본총계']\ntable3 = pd.merge(table1, table2, on='종목코드')\n\ntable3['부채비율']=table3['당기_x']/table3['당기_y']*100\ntable3[table3['부채비율']<100]\n\ndf_sales1 = df4[df4['항목명']=='영업이익']\ndf_sales2 = df4[df4['항목명']=='영업이익(손실)']\ndf_sales = df_sales1.append(df_sales2)"

In [4]:
title_list = []
start_price_list = []   # 액면가
total_price_list = []   # 시가총액
total_stocks_list = []  # 상장주식수
foreign_rate_list = []  # 외국인비율
volume_list = []        # 거래량
per_list = []           # PER
roe_list = []           # ROE

total_sales_list = []  # 매출액
total_asset_list = []  # 자산총계       
total_debt_list = []   # 부채총계
sales_profit_list = [] # 영업이익
sales_increase_list = [] # 매출액증가율
sales_profit_increase_list = [] # 영업이익 증가율

net_income_list = []    # 당기순이익
eps_list = []           # 주당순이익
dividends_list = []     # 보통주배당금
roa_list = []           # ROA
pbr_list = []           # PBR
reserve_rate_list = []  # 유보율

In [6]:
browser = webdriver.Chrome(executable_path='./chromedriver')

for i in range(30):
    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page={}".format(i+1)
    browser.get(url)
    req = browser.page_source
    obj = BeautifulSoup(req, "html.parser")
    
    temp_obj = obj.find('div', {'class':'box_type_l'})
    temp_obj = temp_obj.find('tbody')
    final_obj = temp_obj.find_all('tr', {'onmouseover':'mouseOver(this)'})
    
    for i in range(len(final_obj)):
        title = final_obj[i].find_all('td')[1].text
        start_price = final_obj[i].find_all('td')[5].text.replace(",","")
        total_price = final_obj[i].find_all('td')[6].text.replace(",","")
        total_stocks = final_obj[i].find_all('td')[7].text.replace(",","")
        foreign_rate = final_obj[i].find_all('td')[8].text.replace(",","")
        volume = final_obj[i].find_all('td')[9].text.replace(",","")
        per = final_obj[i].find_all('td')[10].text.replace(",","")
        roe = final_obj[i].find_all('td')[11].text.replace(",","")

        list_append(title, title_list)
        list_append(start_price, start_price_list)
        list_append(total_price, total_price_list)
        list_append(total_stocks, total_stocks_list)
        list_append(foreign_rate, foreign_rate_list)
        list_append(volume, volume_list)
        list_append(per, per_list)
        list_append(roe, roe_list)
        

In [7]:
df1 = pd.DataFrame(list(zip(title_list, 
                           start_price_list, 
                           total_price_list, 
                           total_stocks_list,
                           foreign_rate_list,
                           volume_list,
                           per_list,
                           roe_list)),
                 columns=['종목명', 
                          '액면가', 
                          '시가총액', 
                          '상장주식수', 
                          '외국인비율', 
                          '거래량', 
                          'PER', 
                          'ROE'])

In [8]:
# 거래량 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[1]/td[1]/input').click()
# 외국인 비율 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[3]/td[3]/input').click()
# 상장주식수 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[4]/td[3]/input').click()
# 시가총액 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[1]/td[4]/input').click()
# PER 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[1]/td[6]/input').click()
# ROE 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[2]/td[6]/input').click()

# 매출액 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[4]/td[4]/input').click()
# 자산총계 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[2]/td[4]/input').click()
# 부채총계 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[3]/td[4]/input').click()
# 영업이익 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[1]/td[5]/input').click()
# 메출액 증가율 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[5]/td[4]/input').click()
# 영업이익증가율 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[2]/td[5]/input').click()

# 적용하기 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/div/a[1]/img').click()

In [10]:
for i in range(30):
    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page={}".format(i+1)
    browser.get(url)
    req = browser.page_source
    obj = BeautifulSoup(req, "html.parser")
    
    temp_obj = obj.find('div', {'class':'box_type_l'})
    temp_obj = temp_obj.find('tbody')
    final_obj = temp_obj.find_all('tr', {'onmouseover':'mouseOver(this)'})
    
    for i in range(len(final_obj)):
        title = final_obj[i].find_all('td')[1].text
        total_sales = final_obj[i].find_all('td')[6].text.replace(",","")
        total_asset = final_obj[i].find_all('td')[7].text.replace(",","")
        total_debt = final_obj[i].find_all('td')[8].text.replace(",","")
        sales_profit = final_obj[i].find_all('td')[9].text.replace(",","")
        sales_increase = final_obj[i].find_all('td')[10].text.replace(",","")
        sales_profit_increase = final_obj[i].find_all('td')[11].text.replace(",","")

        list_append(title, title_list)
        list_append(total_sales, total_sales_list)
        list_append(total_asset, total_asset_list)
        list_append(total_debt, total_debt_list)
        list_append(sales_profit, sales_profit_list)
        list_append(sales_increase, sales_increase_list)
        list_append(sales_profit_increase, sales_profit_increase_list)

In [11]:
df2 = pd.DataFrame(list(zip(title_list,
                           total_sales_list, 
                           total_asset_list,
                           total_debt_list,
                           sales_profit_list,
                           sales_increase_list,
                           sales_profit_increase_list)),
                  columns=['종목명',
                           '매출액', 
                           '자산총계', 
                           '부채총계', 
                           '영업이익', 
                           '매출액증가율', 
                           '영업이익증가율'])

In [12]:
# 매출액 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[4]/td[4]/input').click()
# 자산총계 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[2]/td[4]/input').click()
# 부채총계 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[3]/td[4]/input').click()
# 영업이익 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[1]/td[5]/input').click()
# 메출액 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[5]/td[4]/input').click()
# 영업이익증가율 해제
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[2]/td[5]/input').click()



# 당기순이익 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[3]/td[5]/input').click()
# 주당순이익 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[4]/td[5]/input').click()
# 보통주배당금 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[5]/td[5]/input').click()
# ROA 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[3]/td[6]/input').click()
# PBR 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[4]/td[6]/input').click()
# 유보율 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/table/tbody/tr[5]/td[6]/input').click()


# 적용하기 클릭
browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/form/div/div/div/a[1]/img').click()

In [14]:
for i in range(30):
    url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page={}".format(i+1)
    browser.get(url)
    req = browser.page_source
    obj = BeautifulSoup(req, "html.parser")
    
    temp_obj = obj.find('div', {'class':'box_type_l'})
    temp_obj = temp_obj.find('tbody')
    final_obj = temp_obj.find_all('tr', {'onmouseover':'mouseOver(this)'})
    
    for i in range(len(final_obj)):
        title = final_obj[i].find_all('td')[1].text
        net_income = final_obj[i].find_all('td')[6].text.replace(",","")
        eps = final_obj[i].find_all('td')[7].text.replace(",","")
        dividends = final_obj[i].find_all('td')[8].text.replace(",","")
        roa = final_obj[i].find_all('td')[9].text.replace(",","")
        pbr = final_obj[i].find_all('td')[10].text.replace(",","")
        reserve_rate = final_obj[i].find_all('td')[11].text.replace(",","")

        list_append(title, title_list)
        list_append(net_income, net_income_list)
        list_append(eps, eps_list)
        list_append(dividends, dividends_list)
        list_append(roa, roa_list)
        list_append(pbr, pbr_list)
        list_append(reserve_rate, reserve_rate_list)

In [15]:
df3 = pd.DataFrame(list(zip(title_list,
                           total_sales_list, 
                           total_asset_list,
                           total_debt_list,
                           sales_profit_list,
                           sales_increase_list,
                           sales_profit_increase_list)),
                  columns=['종목명',
                           '당기순이익',
                           '주당순이익', 
                           '보통부배당금', 
                           'ROA', 
                           'PBR', 
                           '유보율'])

In [28]:
df1 = df1[df1['ROE'] != 'Null']
df1 = df1[df1['PER'] != 'Null']
df2 = df2[df2['영업이익'] != 'Null']
df3 = df3[df3['PBR'] != 'Null']

In [29]:
result = pd.merge(df1, df2, on='종목명')
result = pd.merge(result, df3, on='종목명')

In [30]:
column_type = {'액면가' : 'int',
               '시가총액' : 'int',
               '상장주식수' : 'int', 
               '외국인비율' : 'float',
               '거래량' : 'int', 
               'PER' : 'float',
               'ROE' : 'float',
               '매출액' : 'int', 
               '자산총계' : 'int',
               '부채총계' : 'int',
               '영업이익' : 'int',
               '매출액증가율' : 'float',
               '영업이익증가율' : 'float', 
               '당기순이익' : 'int',
               '주당순이익' : 'int',
               '보통부배당금' : 'int', 
               'ROA': 'float',
               'PBR': 'float', 
               '유보율': 'float'}

In [31]:
result = result.astype(column_type)
result = result[result['PBR']>0].sort_values(by='PBR')
result['pbr_rank'] = result['PBR'].rank(method='max')
#result.to_csv(r'./result.csv', index=True, header=True)

In [32]:
df = pd.merge(result, magic, left_on='종목명', right_on='회사명')

In [33]:
df['final_rank'] = df['pbr_rank'] + df['magic_rank']

In [34]:
df.sort_values(by='final_rank')

,종목명,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,매출액,자산총계,...,주당순이익,보통부배당금,ROA,PBR,유보율,pbr_rank,회사명,magic,magic_rank,final_rank
15,에이스테크,500,4221,40778,1.53,3349948,-15.52,-11.08,3786,3805,...,3805,2914,27.0,0.34,-79.38,27.0,에이스테크,1.002323,50.0,77.0
47,CSA 코스믹,500,1546,37072,1.59,0,-11.81,-38.05,643,332,...,332,167,-57.0,1.20,-28432.76,71.0,CSA 코스믹,1.668749,14.0,85.0
1,JW홀딩스,500,3662,66452,5.17,296172,-13.34,-20.18,7264,12435,...,12435,8918,27.0,0.14,-91.79,8.0,JW홀딩스,0.813757,93.0,101.0
62,화승알앤에이,500,1062,64561,2.42,186322,5.98,9.62,15520,12116,...,12116,9493,650.0,1.56,20.44,94.0,화승알앤에이,1.075272,41.0,135.0
8,대상,1000,9268,34648,16.93,287807,7.03,11.17,29640,23971,...,23971,13749,1298.0,0.24,8.03,17.0,대상,0.738046,120.0,137.0
24,크리스에프앤씨,1000,2408,11715,0.13,8142,11.56,11.77,2595,2773,...,2773,603,377.0,0.61,-13.51,44.0,크리스에프앤씨,0.791369,103.0,147.0
50,명문제약,500,1641,25921,2.64,686918,-6.59,-24.55,1493,2644,...,2644,1842,-143.0,1.25,-389.54,75.0,명문제약,0.853331,80.0,155.0
94,인디에프,500,1161,58933,0.40,1544672,-32.30,-5.02,2025,1494,...,1494,867,-16.0,2.49,-178.35,140.0,인디에프,1.639103,16.0,156.0
17,한라,5000,1151,38875,2.75,109203,13.52,0.97,13049,15882,...,15882,13271,677.0,0.40,13.06,30.0,한라,0.660414,148.0,178.0
115,BGF리테일,1000,23852,17284,32.09,23331,16.75,26.46,59461,21724,...,21724,15498,1966.0,2.95,3.75,169.0,BGF리테일,1.696939,12.0,181.0


In [35]:
df.to_csv(r'./df.csv', index=True, header=True)